# IMPORT ED AUTENTICAZIONE

In [4]:
import os 
import json
import pprint
import tweepy

In [5]:
#Twitter API credentials
api_key = "your_api_key_here"
api_secret = "your_api_secret_here"
access_token = "your_access_token_here"
access_secret = "your_access_secret_here"
bearer_token = "your_bearer_token_here"

In [6]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')

Authentication completed successfully!


# GESTIONE JSON

In [7]:
data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}


# FOLLOWERS

In [ ]:
avengers = ["Iron_Man", "CaptainAmerica", "Hulk", "thorofficial"]
followers = {}
quantity = 500

for avenger in avengers:
    
    print(f"Processing Avenger {avenger}")
    
    followers_of_avenger = []
    
    for item in tweepy.Cursor(
        api.followers, 
        screen_name=avenger, 
        skip_status=True, 
        include_user_entities=False
    ).items(quantity):
        
        json_data = item._json
        
        found_follower = {}
        found_follower["id"] = json_data["id"]
        found_follower["name"] = json_data["name"]
        found_follower["screen_name"] = json_data["screen_name"]
        found_follower["location"] = json_data["location"]
        
        if found_follower not in followers_of_avenger:
            followers_of_avenger.append(found_follower)
            
    followers[avenger] = followers_of_avenger
    print(f"Found {len(followers[avenger])} followers for avenger {avenger}")
    
serialize_json(data_folder, "avengers_followers.json", followers)

In [9]:
followers = read_json(f"{data_folder}/avengers_followers.json")
# (Pretty) print the first three followers of Iron Man
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(followers['Iron_Man'][:3])

Data read from path: data/avengers_followers.json
[{'id': 1321134178356719616,
  'location': '',
  'name': 'Manas Gajurel',
  'screen_name': 'GajurelManas'},
 {'id': 1321348616561348609,
  'location': '',
  'name': 'Akai',
  'screen_name': 'Akai27111450'},
 {'id': 148821058,
  'location': '',
  'name': 'DeepBlue',
  'screen_name': 'wildcarde'}]


# ESERCIZI

# 1 - Scaricate i dettagli del profilo di Capitan America

In [10]:
avenger = "CaptainAmerica"
raw_data = api.get_user(avenger)
json_data = raw_data._json
profile_data = {}
profile_data["id"] = json_data["id"]
profile_data["name"] = json_data["name"]
profile_data["screen_name"] = json_data["screen_name"]
profile_data["location"] = json_data["location"]
profile_data["description"] = json_data["description"]
profile_data["followers_count"] = json_data["followers_count"]
serialize_json(data_folder, "1_captain_america_profile_data.json", profile_data)
pp.pprint(profile_data)

Data serialized to path: data/1_captain_america_profile_data.json
{'description': 'Just a kid from Brooklyn.',
 'followers_count': 2458213,
 'id': 701615052,
 'location': '',
 'name': 'Captain America',
 'screen_name': 'CaptainAmerica'}


# 2 - Scaricate gli ultimi 10 tweet di Iron Man

In [8]:
avenger = "Iron_Man"
tweets = []
for item in tweepy.Cursor(
        api.user_timeline, 
        screen_name=avenger,
        tweet_mode="extended"
    ).items(10):
    raw_tweet = item._json
    tweet = {}
    tweet["id"]= raw_tweet["id"]
    tweet["created_at"] = raw_tweet["created_at"]
    tweet["full_text"] = raw_tweet["full_text"],
    tweet["favorite_count"]= raw_tweet["favorite_count"]
    tweets.append(tweet)
serialize_json(data_folder, "2_iron_man_recent_tweets.json", tweets)
pp.pprint(tweets)

Data serialized to path: data/2_iron_man_recent_tweets.json
[{'created_at': 'Sat Oct 24 20:00:00 +0000 2020',
  'favorite_count': 3432,
  'full_text': ("It's not called an iconic landing for nothing. What's your "
                'favorite Iron Man moment from Phase 1 of the MCU? '
                'https://t.co/WlWyCXQutU',),
  'id': 1320092701329588226},
 {'created_at': 'Mon Oct 19 20:00:04 +0000 2020',
  'favorite_count': 833,
  'full_text': ("Want to start reading Iron Man? Here's your guide to Armored "
                'Avenger comics on @MarvelUnlimited: https://t.co/xUYqg50yme '
                'https://t.co/NnD3QMqkHB',),
  'id': 1318280777851760640},
 {'created_at': 'Mon Oct 19 17:48:24 +0000 2020',
  'favorite_count': 0,
  'full_text': ('RT @Marvel: 🆕 Access comics on @MarvelUnlimited just three '
                'months after they’re in stores!\n'
                '\n'
                '📚📲 Starting today, read your favorite stories e…',),
  'id': 1318247643076845568},
 {'create

# 3 - Scaricate 20 tweet preferiti da Iron Man

In [9]:
avenger = "Iron_Man"
raw_favorites = api.favorites(
    screen_name=avenger,
    tweet_mode="extended"
)
tweets = []
for raw_tweet in raw_favorites:
    json_tweet = raw_tweet._json
    tweet = {}
    tweet["id"]= json_tweet["id"]
    tweet["created_at"] = json_tweet["created_at"]
    tweet["full_text"] = json_tweet["full_text"],
    tweet["favorite_count"]= json_tweet["favorite_count"]
    tweets.append(tweet)
serialize_json(data_folder, "3_iron_man_favorite_tweets.json", tweets)
pp.pprint(tweets)

Data serialized to path: data/3_iron_man_favorite_tweets.json
[{'created_at': 'Mon Oct 12 21:24:55 +0000 2020',
  'favorite_count': 2423,
  'full_text': ('how it started         how it’s going '
                'https://t.co/tocVizIWNE',),
  'id': 1315765417365790721},
 {'created_at': 'Thu Sep 10 11:21:47 +0000 2020',
  'favorite_count': 33960,
  'full_text': ('The lab of the brilliant Tony Stark arrives! Discover new '
                'powers and continue the fight in the #FortniteNexusWar\n'
                '\n'
                'Explore Iron Man’s Stark Industries on the Island now.\n'
                '\n'
                'Read more: https://t.co/ojVvVRQmDz https://t.co/fbsPFYwVux',),
  'id': 1304017219458916353},
 {'created_at': 'Thu Aug 27 06:13:18 +0000 2020',
  'favorite_count': 7246,
  'full_text': ('The war to save Reality starts now. \n'
                '\n'
                'The champions have arrived on the Island. Join the fight and '
                'take on Galactus.\n'
  

# 4 - Scaricare 1 retwittatore di un ciascuno dei tweet preferiti 

In [7]:
iron_man_favorite_tweets = read_json("data/3_iron_man_favorite_tweets.json")
avenger = "Iron_Man"
retweeters = {}
for tweet in iron_man_favorite_tweets:
    print(f"Processing tweet with id: {tweet['id']}")
    for item in tweepy.Cursor(
            api.retweeters, 
            id=tweet["id"],
        ).items(1):
        print(f"Retweeter ID: {item}")    
        json_data = api.get_user(id=item)._json
        if json_data["id"] not in retweeters.keys():
            profile_data = {}
            profile_data["id"] = json_data["id"]
            profile_data["name"] = json_data["name"]
            profile_data["screen_name"] = json_data["screen_name"]
            profile_data["location"] = json_data["location"]
            profile_data["description"] = json_data["description"]
            profile_data["followers_count"] = json_data["followers_count"]
            retweeters[profile_data["id"]] = profile_data
serialize_json(data_folder, "4_iron_man_favorite_tweets_retweeters.json", retweeters)
pp.pprint(retweeters)
     

Data read from path: data/3_iron_man_favorite_tweets.json
Processing tweet with id: 1315765417365790721
Retweeter ID: 83210357
Processing tweet with id: 1304017219458916353
Retweeter ID: 1282618627846557696
Processing tweet with id: 1298866156086984704
Retweeter ID: 1033359906227679232
Processing tweet with id: 1298864332462346241
Retweeter ID: 997299924
Processing tweet with id: 1279092532350091265
Retweeter ID: 941020920671555584
Processing tweet with id: 1277697093646184448
Retweeter ID: 1051785764
Processing tweet with id: 1276636354173116416
Retweeter ID: 1300218169479569408
Processing tweet with id: 1265780032804278272
Retweeter ID: 735503033845448704
Processing tweet with id: 1254946177717374976
Retweeter ID: 41461775
Processing tweet with id: 1245435808867213312
Retweeter ID: 1279259789277741058
Processing tweet with id: 1215758888751988738
Retweeter ID: 1177308278310744069
Processing tweet with id: 1180539850690301953
Retweeter ID: 1088748236565708800
Processing tweet with id: